In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#seed
numpy.random.seed(7)

In [3]:
#define raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

#create mapping of characters to integers (0-25) and reverse
char_to_int = dict((c,i) for i,c in enumerate(alphabet))
int_to_char = dict((i,c) for i,c in enumerate(alphabet))

In [5]:
#prepare dataset of input to output pairs encoded as integers
seq_length=1
dataX=[]
dataY=[]
for i in range(0,len(alphabet)-seq_length,1):
    seq_in = alphabet[i:i+seq_length]
    seq_out = alphabet[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print(seq_in ,'->', seq_out)

A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


In [10]:
#reshape X to be [samples,time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

In [11]:
#normalize X as LSTM sigmoid needs between 0 & 1
X = X/float(len(alphabet))

In [15]:
#one hot encoding of output
y = np_utils.to_categorical(dataY)

(25, 26)

In [ ]:
#create and fit model
#LSTM state maintained within a batch
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5000, batch_size=len(dataX), verbose=1, shuffle=False)

In [25]:
#summarize performance of the model
scores = model.evaluate(X, y)
print("Accuracy: %.2f%%" %(scores[1]*100))
scores

25/25 [==============================] - 1s 25ms/step
Accuracy: 100.00%


[0.6002871990203857, 1.0]

In [26]:
#demonstrate the model prediction
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/float(len(alphabet))
    prediction = model.predict(x)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in ,'->', result)

['A'] -> B
['B'] -> C
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> S
['S'] -> T
['T'] -> U
['U'] -> V
['V'] -> W
['W'] -> X
['X'] -> Y
['Y'] -> Z


In [27]:
#demonstrate the model predicting random patterns
print("test a random pattern:")
for i in range(0,20):
    pattern_index = numpy.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/float(len(alphabet))
    prediction = model.predict(x)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in ,'->', result)

test a random pattern:
['W'] -> X
['M'] -> N
['B'] -> C
['T'] -> U
['G'] -> H
['C'] -> D
['B'] -> C
['L'] -> M
['S'] -> T
['H'] -> I
['M'] -> N
['Y'] -> Z
['P'] -> Q
['F'] -> G
['I'] -> J
['S'] -> T
['L'] -> M
['Y'] -> Z
['Q'] -> R
['J'] -> K
